In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.metrics import r2_score

In [ ]:
# Helper functions for data cleaning

def diff(a, b):
    return a - b

def drop_col(df):
    return df.drop([
        'OP_CARRIER_FL_NUM', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 
        'CANCELLATION_CODE', 'DISTANCE', 'CARRIER_DELAY','WEATHER_DELAY', 
        'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY','Unnamed: 27'
    ], axis=1)

def remove_na(df):
    return df[df['ACTUAL_ELAPSED_TIME'].notna()]

def total_delay(df):
    df['TOTAL_DELAY'] = df.apply(lambda x: diff(x['ACTUAL_ELAPSED_TIME'], x['CRS_ELAPSED_TIME']), axis=1)
    return df

def to_dt(df):
    df["FL_DATE"] = pd.to_datetime(df["FL_DATE"])
    return df

def to_days(df):
    df["FL_DATE_DAYS"] = df['FL_DATE'].dt.dayofyear
    return df

# run on year files which are a leap year
def clean_leapyear_data(df):
    df = drop_col(df)
    df = remove_na(df)
    df = total_delay(df)
    df = to_dt(df)
    df = df[(df.FL_DATE.dt.month != 2) | (df.FL_DATE.dt.day != 29)]
    df.FL_DATE = df.FL_DATE.apply(lambda x: x.replace(year = x.year - 1))
    df = to_days(df)
    return df

# run on all other files
def clean_data(df):
    df = drop_col(df)
    df = remove_na(df)
    df = total_delay(df)
    df = to_dt(df)
    df = to_days(df)
    return df

In [ ]:
# Import Data sets
df09 = pd.read_csv("./df09.csv")
df10 = pd.read_csv("./df10.csv")
df11 = pd.read_csv("./df11.csv")
df12 = pd.read_csv("./df12.csv")
df13 = pd.read_csv("./df13.csv")
df14 = pd.read_csv("./df14.csv")
df15 = pd.read_csv("./df15.csv")
df16 = pd.read_csv("./df16.csv")
df17 = pd.read_csv("./df17.csv")
df18 = pd.read_csv("./df18.csv")

In [ ]:
df = pd.concat([df09, df10, df11, df12, df13, df14, df15, df16, df17, df18], axis=0, ignore_index=True)
df6 = df[df.OP_CARRIER.isin(['WN', 'DL', 'AA', 'UA'])]

### Airline Codes for Top 4 Airlines in the US
* WN -- Southwest 
* DL -- Delta
* AA -- American 
* UA -- United

In [ ]:
dfWN = df6.loc[df.OP_CARRIER.isin(['WN'])]
dfDL = df6.loc[df.OP_CARRIER.isin(['DL'])]
dfAA = df6.loc[df.OP_CARRIER.isin(['AA'])]
dfUA = df6.loc[df.OP_CARRIER.isin(['UA'])]

In [ ]:
airlines = {
    "Southwest" : dfWN,
    "Delta" : dfDL,
    "Amerian" : dfAA,
    "United" : dfUA
}

def flightStats(df, delay, day):
    flights = df.loc[df.FL_DATE_DAYS == day].TOTAL_DELAY
    totalFlights = flights.count()
    
    delayed = flights.loc[(df.TOTAL_DELAY >= delay)]
    totalDelay = delayed.count()
    
    percentage = 100*(totalDelay/totalFlights) 
    
    delayMean = delayed.mean()
    delayMedian = delayed.median()
    return delayMean, delayMedian, percentage, totalDelay, totalFlights

def createRow(airline, delay, day):
    row = []
    row.append(airline)
    row.append(delay)
    row.append(day)
    a, b, c, d, e = flightStats(airlines[row[0]], row[1], row[2])
    row.append(a)
    row.append(b)
    row.append(c)
    row.append(d)
    row.append(e)
    return row

In [ ]:
delay = 0

dfS = pd.DataFrame(
    columns = ["Airline", "Delay", "Day", "Mean", "Median", 
               "Percentage", "TotalDelay", "TotalFlights"])
for day in range(1, 366):
    for airline in airlines:
        dfS.loc[len(dfS)] = createRow(airline, delay, day)
groups = dfS.groupby('Airline')

In [ ]:
month_starts = [1,32,61,92,122,153,183,214,245,275,306,336,365]
month_names = ['Jan','Feb','Mar','Apr','May','Jun',
               'Jul','Aug','Sep','Oct','Nov','Dec','End'] 
colors = {
    "Amerian" : '#B41FFF',
    "Delta" : '#CE96A6',
    "Southwest" : '#65C819',
    "United" : '#D33643'
}
text = "The lines are a generalization of the data"

In [ ]:
myline = np.linspace(1, 365, 365)
plt.rcParams["figure.figsize"] = (16,12)

for name, group in groups:
    model = np.poly1d(np.polyfit(group.Day, group.Mean, 5))
    
    plt.scatter(group.Day, group.Mean, s=group.Percentage*2.25, marker='.', facecolors=colors[name], alpha=.5)
    plt.plot(myline, model(myline), linewidth=2, c=colors[name], label=name, alpha=1)
    
plt.title(label="Mean Delay of the top 4 Major US Airlines per Day", fontsize=30, loc='left')
plt.xlabel('Day of the year', fontsize=20)
plt.ylabel('Mean Delay in Min', fontsize=20)
plt.text(-15, 17, text, ha='left', wrap=True, fontsize=14)

plt.xticks(month_starts, month_names, rotation=20, fontsize=14)
plt.yticks(fontsize=14)
plt.gca().xaxis.grid(True, linestyle=(5, (10, 3)), linewidth=1)
plt.legend(prop={'size': 16}, loc='upper right')